# Подготовка данных для обучения моделей

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы: 
* https://scikit-learn.org/stable/modules/compose.html#pipeline-chaining-estimators
* https://pytorch.org/docs/stable/data.html 
* https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
* https://codingnomads.com/pytorch-dataset-to-dataloader-using-collate-fn
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann


In [6]:
import torch as th
import numpy as np
th.device("cuda" if th.cuda.is_available() else "cpu")

device(type='cuda')

## Задачи для совместного разбора

1. Создайте синтетический датасет для задачи регрессии и представьте его в виде `torch.utils.data.Dataset`

In [ ]:
from torch.utils.data import Dataset
from sklearn.datasets import make_regression
import torch

class RegressionDataset(Dataset):
    
    def __init__(self, transform = None):
        X,y = make_regression()
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)
        self.transform = transform
        
    def __getitem__(self, index):
        if self.transform is not None:
            return self.transform(self.X[index]), self.y[index]
        return self.X[index], self.y[index]
    
    def __len__(self):
        return len(self.X)

class MyTransform:
    def __init__(self, multiplier):
        self.multiplier = multiplier
    
    def __call__(self, X):
        return X * self.multiplier

In [10]:
dataset = RegressionDataset()
dataset[0]

(tensor([-6.7169e-02, -5.0703e-01,  1.4055e+00, -8.9865e-02, -2.9194e-01,
          2.1143e+00, -8.0012e-03,  1.0317e-02,  8.0073e-01, -8.9563e-01,
         -7.2777e-01,  6.3399e-01, -9.0896e-01, -1.4136e-01, -1.2968e-01,
         -2.7418e-01, -2.2990e-01,  5.8459e-01,  1.6069e+00, -2.7950e+00,
         -7.6166e-01,  4.5926e-01,  6.7884e-04, -7.5148e-01,  3.1208e-02,
          2.8258e-01,  1.4115e+00,  4.2573e-01, -4.8616e-01,  2.3550e+00,
         -5.1228e-02,  4.3058e-01,  3.9294e-01,  1.1067e+00, -5.1540e-01,
         -9.2879e-01,  2.4574e-01, -7.9043e-01, -1.5508e+00,  1.8640e+00,
          1.1370e-01,  1.3618e-01,  8.7996e-01,  1.0186e-01,  2.2435e+00,
         -4.0273e-01,  8.1333e-01,  1.1521e+00,  4.0215e-01, -3.7604e-01,
          1.5468e+00, -1.1742e+00, -9.7380e-01,  5.2857e-01,  1.3514e+00,
         -4.2870e-01, -3.7925e-01, -5.9290e-01, -7.2526e-01, -1.5570e+00,
         -8.1789e-01, -1.1030e+00,  4.1066e-01,  9.6088e-01, -1.9021e+00,
          2.1997e-01,  3.9371e-01,  2.

In [ ]:
%load_ext cudf.pandas
import pandas as pd

In [11]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=16, shuffle=True, drop_last=True)

for X,y in loader:
    print(X.shape, y.shape)

torch.Size([16, 100]) torch.Size([16])
torch.Size([16, 100]) torch.Size([16])
torch.Size([16, 100]) torch.Size([16])
torch.Size([16, 100]) torch.Size([16])
torch.Size([16, 100]) torch.Size([16])
torch.Size([16, 100]) torch.Size([16])


## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Считайте файл `bank-full.csv` ([источник](https://www.kaggle.com/datasets/hariharanpavan/bank-marketing-dataset-analysis-classification)) в виде `pd.DataFrame`.

Опишите класс `BankDatasetBase`. Решение должно удовлетворять следующим критериям:

* класс наследуется от `torch.utils.data.Dataset`;
* при создании объекта в конструктор передается набор данных в виде `pd.DataFrame`;
* объекты класса имеют поля `X` и `y` с признаками и метками соответственно;
* класс реализует интерфейс последовательностей (`__getitem__` + `__len__`);
* `obj[i]` возвращает кортеж, содержащий `i`-ую строку из `obj.X` (серию) и `i`-ую строку из `obj.y` (строку).
    
Создайте объект класса `BankDatasetBase` и продемонстрируйте работоспособность.

- [ ] Проверено на семинаре

In [ ]:
import pandas as pd
import torch as th
class BankDatasetBase(
    # ...
):
    def __init__(self, data: pd.DataFrame) -> None:
        pass

    def __getitem__(self, idx: int) -> tuple[th.Tensor, th.Tensor]:
        pass
    
    def __len__(self) -> int:
        pass

<p class="task" id="2"></p>

2\. Опишите класс `BankDataset`. Решение должно удовлетворять всем критериям из предыдущего задания, а также:
* при создании объекта в конструктор может быть передан необязательные аргументы `transform` и `target_transform`;
* если аргумент `transform` был передан, то при получении `i`-го элемента, нужно вызвать `transform(x)` и вернуть полученный результат.
* если аргумент `target_transform` был передан, то при получении `i`-го элемента, нужно вызвать `target_transform(y)` и вернуть полученный результат.

Создайте объект класса `BankDataset` и продемонстрируйте работоспособность (без передачи `target_transform` и `transform`).

- [ ] Проверено на семинаре

In [ ]:
from typing import Callable


class BankDataset(
    # ...
):
    def __init__(
        self, 
        data: pd.DataFrame, 
        transform: Callable | None = None,
        target_transform: Callable | None = None
    ) -> None:
        pass

    def __getitem__(self, idx: int) -> tuple:
        # x - набор признаков из idx-й строки
        # y - набор признаков из idx-й строки
        # если при создании был передан transform
        # x = transform(x) 
        # если при создании был передан target_transform
        # y = target_transform(y) 
        
    
    def __len__(self) -> int:
        pass

IndentationError: expected an indented block after function definition on line 15 (1835281915.py, line 24)

<p class="task" id="3"></p>

3\. Опишите класс `OrdinalEncoderTransform`. Решение должно удовлетворять следующим критериям:

* при создании объекта в конструктор передаются названия нечисловых столбцов в датасете
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` имеет один параметр (признаки) и возвращает набор признаков, в котором нечисловые характеристики закодированы целыми числами;
* состояние объекта (индексы для кодирования) обновляется в момент очередного вызова `__call__` (т.е. все данные сразу никогда не передаются никакому методу объекта).
    
Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании объект класса `OrdinalEncoderTransform`.

Важно: не создавайте копию класса `BankDataset` с добавленными в него возможностями этого преобразования, используйте композицию. 

- [ ] Проверено на семинаре

<p class="task" id="4"></p>

4\. Опишите класс `LabelEncoderTransform`. Решение должно удовлетворять следующим критериям:

* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` имеет один параметр (строку) и возвращает целое число, соответствующее этой строке;
* состояние объекта (индексы для кодирования) обновляется в момент очередного вызова `__call__` (т.е. все данные сразу никогда не передаются никакому методу объекта).
    
Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании объекта в качестве аргумента `target_transform` объект класса `LabelEncoderTransform`.

Важно: не создавайте копию класса `BankDataset` с добавленными в него возможностями этого преобразования, используйте композицию. 

- [ ] Проверено на семинаре

<p class="task" id="5"></p>

5\. Опишите класс `ToTensor`.  Решение должно удовлетворять следующим критериям:
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает на вход серию или фрейм и возвращает тензор.

Опишите класс `Compose`.  Решение должно удовлетворять следующим критериям:
* при создании объекта в конструктор передается список объектов `transforms`, каждый из которых имеет метод `__call__(x, y)`;
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает имеет параметра (признаки и класс в числовом виде) и и возвращает кортеж, полученный путем последовательного вызова объектов из `transforms`.

Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании преобразования `Compose` список из объектов LabelEncoderTransform и ToTensor.

Важно: не создавайте копию класса `BankDataset` с добавленными в него возможностями этого преобразования, используйте композицию. 

- [ ] Проверено на семинаре

<p class="task" id="6"></p>

6\. Разделите датасет из предыдущего задания на обучающую и тестовую выборку в соотношении 75% на 25%. Создайте объект `DataLoader` для получения пакетов размера 64, полученных из перемешанного обучающего датасета. Кастомизируйте `DataLoader` (используйте аргумент `collate_fn`) таким образом, чтобы пакет признаков был представлен в виде трехмерного тензора размера 64x2x8 (разделите 16 признаков на два тензора по 8). Получите один пакет и выведите на экран размерность тензоров пакета. 



- [ ] Проверено на семинаре